In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv(r"C:\Users\KISHORE\Desktop\Clickstream\train.csv")

In [3]:
df.head()

,year,month,day,order,country,session_id,page1_main_category,colour,location,model_photography,...,page2_clothing_model_A10,page2_clothing_model_A11,page2_clothing_model_A15,page2_clothing_model_A2,page2_clothing_model_A4,page2_clothing_model_A5,page2_clothing_model_B10,page2_clothing_model_B4,page2_clothing_model_P1,page2_clothing_model_others
0,2008,6,22,21,29,15648,3,13,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2008,5,19,6,29,10018,2,13,3,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2008,7,15,2,29,19388,3,9,5,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2008,5,2,2,29,7181,2,2,4,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2008,6,9,16,29,13493,2,9,5,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle

In [5]:
X = df.drop(columns=['price', 'price_2', 'year', 'month', 'day', 'order', 'country', 'session_id'])
y = df['price']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

filepath=r'C:\Users\KISHORE\Desktop\Clickstream\scaler_regg.pkl'
with open(filepath,'wb') as f:
    pickle.dump(scaler, f)

In [7]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'XGBoost Regressor': XGBRegressor()
}

# Dictionary to store results
results = {}

# Train and evaluate models
for name, model in models.items():
    model.fit(X_train_scaled, y_train)  # Training
    y_pred = model.predict(X_test_scaled)  # Prediction

    # Evaluation metrics
    results[name] = {
        'R² Score': r2_score(y_test, y_pred),
        'MAE': mean_absolute_error(y_test, y_pred),
        'MSE': mean_squared_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred))
    }

# Print results
for model, scores in results.items():
    print(f"\n{model} Performance:")
    for metric, value in scores.items():
        print(f"{metric}: {value:.4f}")


Linear Regression Performance:
R² Score: 0.3352
MAE: 8.0635
MSE: 105.3742
RMSE: 10.2652

Ridge Regression Performance:
R² Score: 0.3352
MAE: 8.0635
MSE: 105.3742
RMSE: 10.2652

Lasso Regression Performance:
R² Score: 0.2450
MAE: 9.3034
MSE: 119.6657
RMSE: 10.9392

XGBoost Regressor Performance:
R² Score: 0.9134
MAE: 1.0379
MSE: 13.7316
RMSE: 3.7056


In [8]:
best_model = models['XGBoost Regressor']

# Save the model
filepath=r'C:\Users\KISHORE\Desktop\Clickstream\xgbregressor.pkl'
with open(filepath, 'wb') as f:
    pickle.dump(best_model, f)

In [10]:
from sklearn.model_selection import GridSearchCV
# Define parameter grids
ridge_params = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
lasso_params = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
xgb_params = {
    'n_estimators': [50, 100, 200], 
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# Initialize models
ridge = Ridge()
lasso = Lasso()
xgb = XGBRegressor(random_state=42)

# Grid search for Ridge
ridge_grid = GridSearchCV(ridge, ridge_params, cv=5, scoring='r2', n_jobs=-1)
ridge_grid.fit(X_train_scaled, y_train)
best_ridge = ridge_grid.best_estimator_
ridge_r2 = r2_score(y_test, best_ridge.predict(X_test_scaled))

# Grid search for Lasso
lasso_grid = GridSearchCV(lasso, lasso_params, cv=5, scoring='r2', n_jobs=-1)
lasso_grid.fit(X_train_scaled, y_train)
best_lasso = lasso_grid.best_estimator_
lasso_r2 = r2_score(y_test, best_lasso.predict(X_test_scaled))

# Grid search for XGBoost
xgb_grid = GridSearchCV(xgb, xgb_params, cv=5, scoring='r2', n_jobs=-1)
xgb_grid.fit(X_train_scaled, y_train)
best_xgb = xgb_grid.best_estimator_
xgb_r2 = r2_score(y_test, best_xgb.predict(X_test_scaled))

# Print best parameters and R² scores
print("\nBest Ridge Parameters:", ridge_grid.best_params_)
print("Ridge R² Score:", ridge_r2)

print("\nBest Lasso Parameters:", lasso_grid.best_params_)
print("Lasso R² Score:", lasso_r2)

print("\nBest XGBoost Parameters:", xgb_grid.best_params_)
print("XGBoost R² Score:", xgb_r2)

c:\Users\KISHORE\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.220e+04, tolerance: 1.662e+03
  model = cd_fast.enet_coordinate_descent(



Best Ridge Parameters: {'alpha': 10}
Ridge R² Score: 0.3352085638579463

Best Lasso Parameters: {'alpha': 0.001}
Lasso R² Score: 0.33520967389371303

Best XGBoost Parameters: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200}
XGBoost R² Score: 0.9135315418243408
